In [1]:
# dd
# https://github.com/tulip-control/dd

# PyDDLib
# https://github.com/thiagopbueno/pyddlib/blob/master/pyddlib/bdd.py

# PyEDA
# https://github.com/cjdrake/pyeda/blob/master/doc/source/bdd.rst


# !pip install dd pyeda pyddlib graphviz

In [2]:
import pandas as pd
import numpy as np
from graphviz import Source
import matplotlib.pyplot as plt

# dd

In [5]:
from dd.autoref import BDD

bdd = BDD()
bdd.declare('x1', 'x2', 'x3')

x1 = bdd.var('x1')
x2 = bdd.var('x2')
x3 = bdd.var('x3')

In [6]:
img1 = x1 & ~x2 & x3
img1.to_expr()

'(~ ite(x1, ite(x2, TRUE, (~ x3)), TRUE))'

In [30]:
# (img1 | bdd.exist([f'x1'], img1)).to_expr()

In [31]:
# ( (img1 | bdd.exist([f'x1'], img1)) | bdd.exist([f'x2'], img1) ).to_expr()

In [32]:
# attempts to test enlarged patterns
u = img1

for i in range(1, 4):
    u = u | bdd.exist([f'x{i}'], img1)

img1 = u
img1.to_expr()

'ite(x1, ite(x2, x3, TRUE), (~ ite(x2, TRUE, (~ x3))))'

In [16]:
try:
    bdd.exist(['x4'], img1)
except AssertionError as e:
    print('x4 was not found!')

x4 was not found!


In [17]:
# reset img1
img1 = x1 & ~x2 & x3
img1.to_expr()

'(~ ite(x1, ite(x2, TRUE, (~ x3)), TRUE))'

In [ ]:
'''
(~
    ite(
        x1,
        ite(
            x2,
            TRUE,
            (~ x3)
        ),
        TRUE
    )
)
'''

In [46]:
img2 = ~x1 & x2 & x3
img2.to_expr()

'(~ ite(x1, TRUE, (~ ite(x2, x3, FALSE))))'

In [47]:
(img2 & (~x1 & x2 & ~x3)) == bdd.false

True

In [48]:
dd = img1 | img2
dd.to_expr()

'(~ ite(x1, ite(x2, TRUE, (~ x3)), (~ ite(x2, x3, FALSE))))'

In [ ]:
'''
(~ 
    ite(
        x1,
        ite(
            x2,
            TRUE,
            (~ x3)
        ),
        (~
            ite(
                x2,
                x3,
                FALSE
            )
        )
    )
)
'''

In [52]:
# test true
img3 = ~x1 & x2 & ~x3
(dd & img3) == bdd.false

True

In [53]:
img3 = ~x1 & x2 & x3
(dd & img3) == bdd.false

False

In [ ]:
bdd.dump('test.png', [dd])

# PyDDlib

In [ ]:
from pyddlib.bdd import BDD

# bdd = BDD(-1, None, None, None)

# x1 = bdd.variable(1)
# x2 = bdd.variable(2)
# x3 = bdd.variable(3)

x1 = BDD.variable(1)
x2 = BDD.variable(2)
x3 = BDD.variable(3)

In [ ]:
img1 = x1 & ~x2 & x3
img1

@ (index=1, id=5)
|  - (value=False, id=1)
|  + (index=2, id=4)
|  |  - (index=3, id=3)
|  |  |  - (value=False, id=1)
|  |  |  + (value=True, id=2)
|  |  + (value=False, id=1)

In [ ]:
img2 = ~x1 & x2 & x3
img2

@ (index=1, id=5)
|  - (index=2, id=4)
|  |  - (value=False, id=1)
|  |  + (index=3, id=3)
|  |  |  - (value=False, id=1)
|  |  |  + (value=True, id=2)
|  + (value=False, id=1)

In [ ]:
dd = img1 | img2
dd

@ (index=1, id=6)
|  - (index=2, id=4)
|  |  - (value=False, id=1)
|  |  + (index=3, id=3)
|  |  |  - (value=False, id=1)
|  |  |  + (value=True, id=2)
|  + (index=2, id=5)
|  |  - (index=3, id=3)
|  |  |  - (value=False, id=1)
|  |  |  + (value=True, id=2)
|  |  + (value=False, id=1)

In [ ]:
dd.is_zero(), dd.is_one()

(False, False)

In [ ]:
img3 = ~x1 & x2 & ~x3
img3

@ (index=1, id=5)
|  - (index=2, id=4)
|  |  - (value=False, id=1)
|  |  + (index=3, id=3)
|  |  |  - (value=True, id=2)
|  |  |  + (value=False, id=1)
|  + (value=False, id=1)

In [ ]:
img3 = ~x1 & x2 & ~x3
(dd & img3).is_zero()

True

In [ ]:
img3 = ~x1 & x2 & x3
(dd & img3).is_zero()

False

In [ ]:
img3 = x1
(dd & img3).is_zero()

False

In [ ]:
img3 = ~x1 & ~x2
(dd & img3).is_zero()

True

In [ ]:
img3 = ~x1 & ~x3
(dd & img3).is_zero()

True

In [ ]:
dd & img3

@ (value=False, id=1)

# PyEDA

In [4]:
# !pip3 install pyeda

In [28]:
from pyeda.inter import *
from pyeda.boolalg.bdd import BDDZERO, BDDONE
from pyeda.boolalg.expr import ITE

x1, x2, x3 = map(bddvar, 'x1 x2 x3'.split())

In [12]:
(~x1).is_one(), (~x1).is_zero()

(False, False)

In [14]:
dd = BDDZERO

img1 = x1 & ~x2 & x3
dd |= img1

img2 = ~x1 & x2 & x3
dd |= img2

In [17]:
img3 = ~x1 & x2 & ~x3
(dd & img3) == BDDZERO

True

In [19]:
img4 = ~x1 & x2 & x3
(dd & img4) == BDDZERO

False

# Testing: how many vatiables till BDD return accuracy 100%?

## Function Parts

In [6]:
from itertools import product

[ *product(['x1', '~x1'], ['x2', '~x2'], ['x3', '~x3']) ]

[('x1', 'x2', 'x3'),
 ('x1', 'x2', '~x3'),
 ('x1', '~x2', 'x3'),
 ('x1', '~x2', '~x3'),
 ('~x1', 'x2', 'x3'),
 ('~x1', 'x2', '~x3'),
 ('~x1', '~x2', 'x3'),
 ('~x1', '~x2', '~x3')]

In [57]:
from dd.autoref import BDD

bdd = BDD()

num_vars = 10
num_combs = np.power(2, num_vars)
print( f"expexted number of combinations is {np.power(2, num_vars)}" ) 

vars = [ f'x{v}' for v in range(num_vars)]
bdd.declare(*vars)

vars = [[bdd.var(v), ~bdd.var(v)] for v in vars]

expexted number of combinations is 1024


In [58]:
(vars[0][0] & vars[1][1]).to_expr()

'(~ ite(x0, x1, TRUE))'

In [59]:
for comb in product(*vars):
    print( np.bitwise_and.reduce(comb).to_expr() )

    break

ite(x0, ite(x1, ite(x2, ite(x3, ite(x4, ite(x5, ite(x6, ite(x7, ite(x8, x9, FALSE), FALSE), FALSE), FALSE), FALSE), FALSE), FALSE), FALSE), FALSE)


In [60]:
# initiate BDD
dd = bdd.false
add_combs = 200

for _, comb in zip(range(add_combs), product(*vars)):
    # print( np.bitwise_and.reduce(comb).to_expr() )
    dd |= np.bitwise_and.reduce(comb)

In [65]:
# test all combs
total = 0

def test_expr(expr1, expr2):
    return (expr1 & expr2) == bdd.false

for comb in product(*vars):
    total += test_expr( dd, np.bitwise_and.reduce(comb) )

total, num_combs, total / num_combs

(824, 1024, 0.8046875)

## Function

In [30]:
# all together
def run_dd_test(num_vars=5, perc=0.1):
    '''
    Test 2**num_vars vraiables
    '''

    from itertools import product
    from dd.autoref import BDD

    bdd = BDD()
    num_combs = np.power(2, num_vars)
    print( f"num vars: {num_vars} | expexted combinations is {np.power(2, num_vars):,}" ) 
    
    # add variables to BDD
    vars = [ f'x{v}' for v in range(num_vars)]
    bdd.declare(*vars)
    
    # generate negeation for all variables
    vars = [[bdd.var(v), ~bdd.var(v)] for v in vars]

    # initiate BDD
    dd = bdd.false
    # add only numper small amount of expressions
    add_combs = int(num_combs * perc)

    # add only some combinations to the BDD
    for _, comb in zip(range(add_combs), product(*vars)):
        dd |= np.bitwise_and.reduce(comb)

    # test all combs
    total = 0

    def test_expr_dd(expr1, expr2):
        '''if comb is not found return True'''
        return not ( (expr1 & expr2) == bdd.false )

    # test all combinations against the added ones
    for comb in product(*vars):
        total += test_expr_dd( dd, np.bitwise_and.reduce(comb) )
    
    # delete variables
    del BDD, bdd, dd, vars

    print(f'perc: {perc} | total: {total} | num_combs: {num_combs} | accuracy: {total / num_combs}')
    print('-' * 100)

    return total, num_combs, total / num_combs

In [39]:
%%time
results = {}
for n in range(5, 16):
    results[n] = run_dd_test(n)

num vars: 5 | expexted combinations is 32
perc: 0.1 | total: 3 | num_combs: 32 | accuracy: 0.09375
----------------------------------------------------------------------------------------------------
num vars: 6 | expexted combinations is 64
perc: 0.1 | total: 6 | num_combs: 64 | accuracy: 0.09375
----------------------------------------------------------------------------------------------------
num vars: 7 | expexted combinations is 128
perc: 0.1 | total: 12 | num_combs: 128 | accuracy: 0.09375
----------------------------------------------------------------------------------------------------
num vars: 8 | expexted combinations is 256
perc: 0.1 | total: 25 | num_combs: 256 | accuracy: 0.09765625
----------------------------------------------------------------------------------------------------
num vars: 9 | expexted combinations is 512
perc: 0.1 | total: 51 | num_combs: 512 | accuracy: 0.099609375
-------------------------------------------------------------------------------------

In [40]:
results

{5: (3, 32, 0.09375),
 6: (6, 64, 0.09375),
 7: (12, 128, 0.09375),
 8: (25, 256, 0.09765625),
 9: (51, 512, 0.099609375),
 10: (102, 1024, 0.099609375),
 11: (204, 2048, 0.099609375),
 12: (409, 4096, 0.099853515625),
 13: (819, 8192, 0.0999755859375),
 14: (1638, 16384, 0.0999755859375),
 15: (3276, 32768, 0.0999755859375)}

In [33]:
%%time
results2 = {}

for p in np.arange(0.1, 0.91, 0.1):
    p = p.round(2)
    results2[p] = run_dd_test(10, p)

num vars: 10 | expexted combinations is 1,024
perc: 0.1 | total: 102 | num_combs: 1024 | accuracy: 0.099609375
----------------------------------------------------------------------------------------------------
num vars: 10 | expexted combinations is 1,024
perc: 0.2 | total: 204 | num_combs: 1024 | accuracy: 0.19921875
----------------------------------------------------------------------------------------------------
num vars: 10 | expexted combinations is 1,024
perc: 0.3 | total: 307 | num_combs: 1024 | accuracy: 0.2998046875
----------------------------------------------------------------------------------------------------
num vars: 10 | expexted combinations is 1,024
perc: 0.4 | total: 409 | num_combs: 1024 | accuracy: 0.3994140625
----------------------------------------------------------------------------------------------------
num vars: 10 | expexted combinations is 1,024
perc: 0.5 | total: 512 | num_combs: 1024 | accuracy: 0.5
------------------------------------------------

In [34]:
results2

{0.1: (102, 1024, 0.099609375),
 0.2: (204, 1024, 0.19921875),
 0.3: (307, 1024, 0.2998046875),
 0.4: (409, 1024, 0.3994140625),
 0.5: (512, 1024, 0.5),
 0.6: (614, 1024, 0.599609375),
 0.7: (716, 1024, 0.69921875),
 0.8: (819, 1024, 0.7998046875),
 0.9: (921, 1024, 0.8994140625)}